## Import thư viện

In [63]:
import torch
import torch.nn as nn
from transformers import DistilBertTokenizer
import re
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from underthesea import word_tokenize


## Cấu hình tùy chọn cho Edge

In [64]:
def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = get_device()
print(f"Thiết bị sử dụng: {device}")

def remove_emoji(text):
    for emoji in UNICODE_EMOJI.values():
        text = text.replace(emoji, "")
    for emoticon in EMOTICONS_EMO.values():
        text = text.replace(emoticon, "")
    return text

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', ' <num> ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r"[!@#$\[\]()']", "", text)

    # Load stopwords once and use set for faster lookup
    with open('vietnamese-stopwords.txt', "r", encoding="utf-8") as f:
        stopwords = set(f.read().split("\n"))
    
    words = word_tokenize(text)
    text = " ".join(word for word in words if word not in stopwords)
    return text


Thiết bị sử dụng: cpu


In [65]:
def preprocess_input(sentence, tokenizer, max_length=128):
    sentence = remove_emoji(sentence)
    sentence = clean_text(sentence)
    tokens = tokenizer(sentence, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
    return tokens['input_ids']

In [66]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, max_length):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size * 2, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, output_size)
        )
        
    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Take the last output of the LSTM
        return out


In [67]:
def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    print(f"Mô hình đã được nạp từ {path}")

# Hàm kiểm tra mô hình
def test_model(model, tokenizer, sentence, device):
    model.eval()
    input_ids = preprocess_input(sentence, tokenizer).to(device)
    with torch.no_grad():
        outputs = model(input_ids)
        _, predicted_label = torch.max(outputs, 1)
    label_map = {0: "Tích cực (POS)", 1: "Trung tính (NEU)", 2: "Tiêu cực (NEG)"}
    return label_map[predicted_label.item()]

In [68]:
checkpoint = 'distilbert-base-multilingual-cased'
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)

In [69]:
vocab_size = tokenizer.vocab_size
embedding_dim = 128
hidden_size = 64
output_size = 3
max_length = 128

In [70]:
model = LSTMModel(vocab_size, embedding_dim, hidden_size, output_size, max_length).to(device)


In [71]:
best_model_path = "./model/best_lstm_model.pth"
load_model(model, best_model_path, device)

Mô hình đã được nạp từ ./model/best_lstm_model.pth


C:\Users\ASUS\AppData\Local\Temp\ipykernel_28852\3551613524.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=device))


In [79]:
# Dự đoán với các câu đầu vào
test_sentences = [
    '''Tớ thất vọng về cậu quá''',
    '''cái này thật là tệ hại!''',
    '''Sản phẩm này tốt'''
]

print("\nKết quả thử nghiệm:")
for sentence in test_sentences:
    prediction = test_model(model, tokenizer, sentence, device)
    print(f"Câu: \"{sentence}\" --> Dự đoán: {prediction}")


Kết quả thử nghiệm:
Câu: "Tớ thất vọng về cậu quá" --> Dự đoán: Tiêu cực (NEG)
Câu: "cái này thật là tệ hại!" --> Dự đoán: Tiêu cực (NEG)
Câu: "Sản phẩm này tốt" --> Dự đoán: Tích cực (POS)
